In [1]:
import os
import re
import torch

from openai import OpenAI
from dataclasses import dataclass
from vllm import LLM, SamplingParams
from typing import List, Dict, Optional
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/pervinco/LLM-tutorials/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-23 08:27:10,426	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from dotenv import load_dotenv
load_dotenv("../keys.env")
api_key = os.getenv('GRAVY_LAB_OPENAI')

client = OpenAI(api_key=api_key)

In [3]:
def generate_completion(prompt: str, model: str = "gpt-3.5-turbo", max_tokens: int = 100) -> str:
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0
    )
    return completion.choices[0].message.content


# 프롬프트 입력
prompt = '아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?'

# 응답 생성
response = generate_completion(prompt)
print(f"응답: {response}")

응답: 소년의 아버지인 외과의사가 말한 대사를 해석하면, "나는 수술을 할 수 없어! 이 아이는 내 아들이니까!"라는 뜻입니다. 따라서, 이 외과의사는 소년의 아버지입니다.


In [4]:
# model = LLM(model="meta-llama/Llama-3.2-1B", dtype="float16")
# prompt = '아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?'
# model.generate(prompt, use_tqdm=False)[0].outputs[0].text

In [5]:
import re
import torch
from dataclasses import dataclass
from vllm import LLM, SamplingParams
from typing import List, Dict, Optional
from transformers import AutoModelForCausalLM, AutoTokenizer


In [6]:
@dataclass
class Path:
    reasoning_text: str
    score: float
    answer_span: str
    num_path: int

@dataclass
class DecodingInfo:
    question: str
    paths: List[Path]

In [7]:
def print_output_details(outputs):
    """
    outputs의 각 요소를 자세히 출력하는 함수
    """
    for i, output in enumerate(outputs, 1):
        print(f"\n{'='*50}")
        print(f"Output #{i}")
        print(f"{'='*50}")
        
        # 1. 기본 정보
        print("\n[기본 정보]")
        print(f"Request ID: {output.request_id}")
        print(f"Prompt: {output.prompt}")
        print(f"Finished: {output.finished}")
        
        # 2. 생성된 텍스트
        print("\n[생성된 텍스트]")
        for out in output.outputs:
            print(f"Index: {out.index}")
            print(f"Text: {out.text}")
            print(f"Cumulative LogProb: {out.cumulative_logprob:.4f}")
            
        # 3. 토큰별 상세 정보
        print("\n[토큰별 상세 정보]")
        print(f"{'Token':^20} | {'LogProb':^10} | {'Alternative Token':^20} | {'Alt LogProb':^10}")
        print("-" * 65)
        
        for out in output.outputs:
            for logprob_dict in out.logprobs:
                for token_id, prob_info in logprob_dict.items():
                    # 메인 토큰 정보
                    token = prob_info.decoded_token
                    if token == '\n': token = '\\n'
                    if token == '\t': token = '\\t'
                    logprob = prob_info.logprob
                    
                    # 대안 토큰이 있는 경우 (항상 2개의 토큰이 있다고 가정)
                    alt_token = ""
                    alt_logprob = ""
                    
                    print(f"{token:^20} | {logprob:^10.4f} | {alt_token:^20} | {alt_logprob:^10}")
        
        print("\n")

# 사용 예시:
# outputs = decoder.generate_paths(prompts)
# print_output_details(outputs)

In [8]:
class CoTDecoder:
    """
    논문에서는 greedy decoding 대신 top-k를 이용하여 다양한 경로를 탐색하는 것을 권장합니다.
    특히 각 경로에서 어떻게 생각하는지 평가할 수 있도록 여러 경로에서 다양한 토큰을 샘플링 해야한다고 설명합니다.
    """
    def __init__(self, 
                 model_name: str, ## 사용할 모델 이름
                 device: str = 'cuda', 
                 max_new_tokens: int = 100, ## 생성할 토큰의 최대 길이
                 topk: int = 5, ## 각 질문에 대해 탐색할 초기 토큰 개수
                 stop: List[str] = ['\n\n질문', '질문', 'Q:', '\n\nQ:', '\n\nExercise'], ## 생성이 멈추는 특정 단어 리스트
                 prompt: str = '', ## 프롬프트 텍스트
                 pattern: str = r'[가-힣a-zA-Z0-9\s]+'): ## 답변에서 추출할 텍스트 패턴(정규표현식)
        
        self.model = LLM(model=model_name, dtype='float16')
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = device
        self.max_new_tokens = max_new_tokens
        self.stop = stop
        self.topk = topk
        self.model.llm_engine.model_config.max_logprobs = self.topk + 1
        self.prompt = prompt
        self.pattern = pattern


    def format_prompt(self, raw_prompt: str) -> str:
        return f'질문:{raw_prompt}\n답변:{self.prompt}'
    

    @torch.inference_mode()
    def get_first_topk_tokens(self, prompt: str) -> Dict[str, List]:
        """
        1단계 :  top-k 토큰 선택
        formatted_prompt가 입력되어 모델이 예측하는 가장 가능성이 높은 top-k 단어를 계산
          - n=1 : 동일한 프롬프트에서 몇 개의 결과(문장)를 생성하고 반환할 것인가? ex) n=5인 경우 하나의 입력에 대해 5개의 서로 다른 문장을 생성한다.
          - top_p : 확률이 상위 100%에 포함되는 모든 단어를 고려. 단계별 토큰들의 확률 누적 합이 p를 넘지 않는 단어들만 선택한다.
          - max_tokens : 모델이 출력할 문장의 최대 길이.
          - logprobs : 상위 k개 단어의 로그 확률값 반환.

        - output : 누적합이 top_p=1 이 되기 이전까지의 토큰들
        """
        sampling_params = SamplingParams(n=1, 
                                         temperature=0, 
                                         top_p=1, 
                                         max_tokens=1, 
                                         logprobs=self.topk, 
                                         stop=self.stop)

        # 모델이 입력된 prompt을 기준으로 prompt 뒤에 올 10개의 단어를 예측합니다. 
        outputs = self.model.generate(prompt, sampling_params, use_tqdm=False)[0].outputs[0].logprobs[0]

        # decoded는 "그", "\n\n", "소", "아이", "이", "아", "어" 등의 단어가 생성되어 저장되어 있습니다. 
        # probs는 -2.064455270767212, -3.392580270767212 등의 로그 확률이 저정되어 있습니다. 
        # token_id는 당연히 token_id가 저장되어 있습니다. 
        topk_tokens = {'decoded': [], 'probs': [], 'token_id': [], 'logprobs': []}
        for token_id, logprob_obj in outputs.items():
            topk_tokens['logprobs'].append({token_id: logprob_obj})
            topk_tokens['decoded'].append(logprob_obj.decoded_token)
            topk_tokens['probs'].append(logprob_obj.logprob)
            topk_tokens['token_id'].append(token_id)

        # 로그 확률을 실제 확률로 변환합니다. 
        topk_tokens['probs'] = torch.exp(torch.tensor(topk_tokens['probs'])).tolist()

        return topk_tokens
    

    @torch.inference_mode()
    def generate_paths(self, prompts: List[str]) -> Dict[int, Dict]:
        """
        topk개의 토큰을 기반으로 다양한 경로를 생성해야 한다. 그 과정을 코드로 구현.

        input : ['질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?\n답변:아이', ...]
        max_tokens = self.max_new_tokens : 최대 self.max_new_tokens인 길이의 문장이 생성된다.

        generate 메서드는 배치 처리를 지원하므로, prompts의 각 원소는 독립적인 입력으로 간주되어 원소별 독립적인 답변을 추론하고 생성하게 된다.
        """
        sampling_params = SamplingParams(n=1, temperature=0, top_p=1, max_tokens=self.max_new_tokens, logprobs=2, stop=self.stop)
        
        return self.model.generate(prompts, sampling_params, use_tqdm=False)
    

    # 질문과 Reasoning의 유사도 계산하는 함수
    def calculate_question_similarity(self, question: str, reasoning: str) -> float:
        """ 질문과 reasoning 간의 유사도를 계산하는 간단한 함수. 유사도가 높으면 패널티를 부여한다 """
        question_words = set(question.split())
        reasoning_words = set(reasoning.split())
        
        # 질문과 reasoning 간에 공통된 단어의 비율 계산
        common_words = question_words.intersection(reasoning_words)
        similarity = len(common_words) / len(question_words) if question_words else 0
        
        return similarity
    

    def calculate_score(self, prompt: str, topk_tokens: Dict, outputs: Dict) -> DecodingInfo:
        """
        모델 출력과 top-k 토큰 기반 reasoning 결과를 분석하여, 각 경로의 점수를 계산하고 정보를 구조화하는 함수.
        top_k 경로 중 어떤 경로가 가장 적절한지 평가한다.
          - question: 원래 입력 질문.
          - paths: 각 경로(topk_tokens의 각 토큰)별 결과와 점수를 포함한 리스트.
        """
        paths = []
        for k, output in enumerate(outputs):
            print("=" * 70)
            ## 각 outputs 항목에 대해, 모델이 생성한 topk_tokens의 k번째 토큰과 모델 출력 텍스트를 이어붙여 reasoning(추론 결과)을 만든다.
            reasoning = topk_tokens['decoded'][k] + output.outputs[0].text
            reasoning = reasoning.strip() ## 불필요한 공백 제거
            print(f"topk_tokens : {topk_tokens['decoded'][k]}, output : {output.outputs[0].text}")
            print(f"reasoning : {reasoning}")
            
            ## 질문과 reasoning 간의 유사도를 계산 (간단한 방식으로 질문이 포함되었는지 확인)
            ## calculate_question_similarity는 질문과 reasoning에 포함된 공통 단어의 비율을 계산하여 간단한 유사도를 반환. 이 유사도는 패널티를 적용하는 데 사용.
            question_similarity = self.calculate_question_similarity(prompt, reasoning)
            
            ## reasoning을 토크나이저로 인코딩하여 토큰 정보와 offset(mapping) 정보를 얻는다.
            encode = self.tokenizer(reasoning, return_offsets_mapping=True) ## 각 토큰이 원래 텍스트에서 차지하는 **시작 및 끝 위치(offset)**를 반환
            answer_span = re.findall(self.pattern, reasoning) ## reasoning에서 정규표현식(self.pattern)과 일치하는 텍스트를 찾는다.
            print(f"encode : {encode}")
            print(f"answer_span : {answer_span}")
            """
            예시
                pattern = r'[가-힣a-zA-Z0-9\s]+'
                reasoning = "이 외과의사는 소년의 아버지입니다."
                answer_span = re.findall(pattern, reasoning)

                ['이 외과의사는 소년의 아버지입니다']
            """
            
            ## answer_span이 reasoning에서 정확히 어디에 있는지 위치를 확인한다.
            score = 0
            if len(answer_span):
                answer_span = answer_span[-1]
                last_pattern_span = (reasoning.rfind(answer_span), reasoning.rfind(answer_span) + len(answer_span)) ## answer_span이 reasoning에서 마지막으로 등장하는 위치를 찾는다.
                print(f"last_pattern_span : {last_pattern_span}")
                """
                reasoning: "이 외과의사는 소년의 아버지입니다."
                answer_span: "소년의 아버지"
                last_pattern_span = (8, 14)  --> 8은 "소년의"의 시작 위치, 14는 "아버지"의 끝 위치
                """

                ## answer_span에 해당하는 token_id 찾기
                idx_answer = [i for i, span in enumerate(encode.offset_mapping)
                            if (span[0] >= last_pattern_span[0] and span[1] <= last_pattern_span[1]) or
                                (span[0] <= last_pattern_span[0] and span[1] >= last_pattern_span[1]) or
                                (span[0] <= last_pattern_span[0] and span[1] > last_pattern_span[0])]

                token_id = [encode.input_ids[idx] for idx in idx_answer] ## 위에서 찾은 idx_answer를 바탕으로, encode.input_ids에서 해당 토큰 ID를 가져온다.
                print(f"token_id : {token_id}")
                output.outputs[0].logprobs.insert(0, topk_tokens['logprobs'][k])

                filtered_answer = [output for i, output in enumerate(output.outputs[0].logprobs) if i in idx_answer] ## 모델이 생성한 각 토큰의 로그 확률 정보에서, idx_answer에 해당하는 토큰들만 추출
                print(f"filtered_answer : {filtered_answer}")

                ## 로그 확률을 지수 함수(exp)로 변환하여 실제 확률 값으로 바꾼다.
                sum_answer_span_probs = 0
                for logprob_dict in filtered_answer:
                    logprob_list = list(logprob_dict.items())
                    if len(logprob_list) == 2:
                        prob_diff = (torch.exp(torch.tensor([logprob_list[0][1].logprob])) - torch.exp(torch.tensor([logprob_list[1][1].logprob]))).item()
                    else:
                        prob_diff = torch.exp(torch.tensor([logprob_list[0][1].logprob])).item()
                    sum_answer_span_probs += prob_diff
                
                ## 질문과 비슷한 답변일 경우 페널티 적용
                if question_similarity > 0.5:  # 질문과의 유사도가 높을수록 점수를 낮추기 위해 0.5 이상의 유사도에 패널티 적용
                    sum_answer_span_probs *= (1 - question_similarity)  # 유사도가 높을수록 점수를 감소시킴

                ## 최종 점수 계산.
                score = 0 if len(filtered_answer) == 0 else sum_answer_span_probs / len(filtered_answer)
                answer_span = self.tokenizer.decode(token_id, skip_special_tokens=True).strip()
            else:
                answer_span = '|<NotFound>|'

            paths.append(Path(reasoning_text=reasoning, 
                            score=score,
                            answer_span=answer_span,
                            num_path=k))

        return DecodingInfo(question=prompt, paths=paths)


    ## chain of thought 탐색
    def search_cots(self, raw_prompt: str) -> DecodingInfo:
        formatted_prompt = self.format_prompt(raw_prompt) ## 질문 : ..., 답변 : ... 형식으로 변경
        print(f"format_prompt : {formatted_prompt}\n")

        ## 질문에 이어서 나올 가능성이 높은 top-k 토큰들을 생성(단어를 top_k개만큼 생성)하고 token_id, 생성된 토큰, 확률값을 저장한 topk_token을 생성.
        ## ['이', '아이', '아', ' 이', '소']
        topk_tokens = self.get_first_topk_tokens(formatted_prompt)
        print(f"top k tokens")
        for k, v in topk_tokens.items():
            print(k, v)
        print("\n")
        
        ## 생성된 topk 토큰과 질문을 각각 이어 붙여주고 prompts라는 리스트에 저장. 
        ## '질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?\n답변:이', 
        ## '질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?\n답변:아이'
        prompts = [formatted_prompt + token for token in topk_tokens['decoded']] ## top_k가 정답에 이어 붙어진 5개의 독립적인 프롬프트가 리스트 형태로 연결되어 프롬프트를 구성함.
        print(f"prompts")
        for prompt in prompts:
            print(prompt)
        print("\n")
        
        outputs = self.generate_paths(prompts)
        # print_output_details(outputs)
        print(f"outputs")
        for output in outputs:
            print(output)
        print("\n")
        
        return self.calculate_score(raw_prompt, topk_tokens, outputs)

In [9]:
model_name = "meta-llama/Llama-3.2-1B"
decoder = CoTDecoder(model_name)

WARNING 12-23 08:27:13 config.py:2171] Casting torch.bfloat16 to torch.float16.
INFO 12-23 08:27:16 config.py:478] This model supports multiple tasks: {'embed', 'score', 'reward', 'classify', 'generate'}. Defaulting to 'generate'.
WARNING 12-23 08:27:16 arg_utils.py:1086] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-23 08:27:16 config.py:1364] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 12-23 08:27:16 llm_engine.py:249] Initializing an LLM engine (v0.6.5) with config: model='meta-llama/Llama-3.2-1B', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=N

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.32it/s]



INFO 12-23 08:27:19 model_runner.py:1097] Loading model weights took 2.3185 GB
INFO 12-23 08:27:20 worker.py:241] Memory profiling takes 0.46 seconds
INFO 12-23 08:27:20 worker.py:241] the current vLLM instance can use total_gpu_memory (23.64GiB) x gpu_memory_utilization (0.90) = 21.28GiB
INFO 12-23 08:27:20 worker.py:241] model weights take 2.32GiB; non_torch_memory takes 0.11GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 17.67GiB.
INFO 12-23 08:27:20 gpu_executor.py:76] # GPU blocks: 36197, # CPU blocks: 8192
INFO 12-23 08:27:20 gpu_executor.py:80] Maximum concurrency for 131072 tokens per request: 4.42x
INFO 12-23 08:27:21 model_runner.py:1413] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-23 08:27:21 model_runner.py:1417] If out-of-memory error occurs during cudagraph captu

In [10]:
prompt = '아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?'
# prompt = "I have 3 apples, my dad has 2 more apples than me, how many apples do we have in total?"
result = decoder.search_cots(prompt)

print(f"Question: {result.question}")
for path in result.paths:
    print(f"Path {path.num_path}:")
    print(f"  Reasoning: {path.reasoning_text}")
    print(f"  Answer: {path.answer_span}")
    print(f"  Score: {path.score:.4f}")
    print()

format_prompt : 질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?
답변:

top k tokens
decoded ['이', '아이', '아', ' 이', '소']
probs [0.09347356110811234, 0.06275518238544464, 0.05003279075026512, 0.03631991147994995, 0.02741570770740509]
token_id [13094, 114714, 54059, 23955, 44690]
logprobs [{13094: Logprob(logprob=-2.3700766563415527, rank=1, decoded_token='이')}, {114714: Logprob(logprob=-2.7685141563415527, rank=2, decoded_token='아이')}, {54059: Logprob(logprob=-2.9950766563415527, rank=3, decoded_token='아')}, {23955: Logprob(logprob=-3.3153891563415527, rank=4, decoded_token=' 이')}, {44690: Logprob(logprob=-3.5966391563415527, rank=5, decoded_token='소')}]


prompts
질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?
답변:이
질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?
답변:아이
질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?
답변:아
질문:아이의 아버지인 외과의사가 말했어. "난 수술 못해! 이 아이는 내 아들이라고!" 이 외과의사는 소년에게 누구일까요?
답변: 이
질문:아이의 아버지인 